In [1]:
from cuqi.distribution import DistributionGallery, Gaussian, JointDistribution
from cuqi.testproblem import Poisson1D
from cuqi.problem import BayesianProblem
import cuqi
import inspect
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH, ULA, MALA, NUTS
import time
import scipy.stats as sps
from scipy.stats import gaussian_kde
import pandas as pd
import arviz
import cuqi
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
from utilities import TableAutomization as TA
import benchmarksClass as BC 

In [2]:
# The donut distribution 
target_donut = DistributionGallery("donut")

# The Poisson1D Bayesian problem
dim = 201
L = np.pi

xs = np.array([0.2, 0.4, 0.6, 0.8])*L
ws = 0.8
sigma_s = 0.05
def f(t):
    s = np.zeros(dim-1)
    for i in range(4):
        s += ws * sps.norm.pdf(t, loc=xs[i], scale=sigma_s)
    return s

A, _, _ = Poisson1D(dim=dim, 
                    endpoint=L,
                    field_type='KL',
                    field_params={'num_modes': 10} ,
                    map=lambda x: np.exp(x), 
                    source=f).get_components()
sigma_x = 30
x = Gaussian(0, sigma_x**2, geometry=A.domain_geometry)
np.random.seed(12)
x_true = x.sample()
sigma_y = np.sqrt(0.001)
y = Gaussian(A(x), sigma_y**2, geometry=A.range_geometry)
y_obs = y(x=x_true).sample()
joint = JointDistribution(y, x)
target_poisson = joint(y=y_obs)

In [3]:
[sampler for sampler in dir(cuqi.sampler) if not sampler.startswith('_')]

['CWMH',
 'Conjugate',
 'ConjugateApprox',
 'Gibbs',
 'LinearRTO',
 'MALA',
 'MH',
 'NUTS',
 'ProposalBasedSampler',
 'RegularizedLinearRTO',
 'Sampler',
 'UGLA',
 'ULA',
 'pCN']

In [4]:
Ns_MH_fixed = 10000 # Number of samples
Nb_MH_fixed = 0 # Number of burn-in samples
scale_MH_fixed = 0.05 # Fixed step size "scale"
MH_sampler = MH(target=target_donut, scale=scale_MH_fixed, x0=np.array([0,0]))

MH_fixed_samples = MH_sampler.sample(Ns_MH_fixed, Nb_MH_fixed) #MH fixed sampling

Sample 10000 / 10000

Average acceptance rate: 0.8742 



In [5]:
MH_fixed_samples = MH_sampler.sample(Ns_MH_fixed, Nb_MH_fixed) #MH fixed sampling
a = MH_fixed_samples.compute_ess()
b = np.mean(a)
print(a)
print(b)

Sample 10000 / 10000

Average acceptance rate: 0.8727 

[7.46127098 6.33857287]
6.899921929102696


In [60]:
arviz.mcse(MH_fixed_samples.to_arviz_inferencedata())['v0']

<xarray.DataArray 'v0' ()> Size: 8B
array(0.21114119)

In [12]:
x= TA.precompute_samples(target_donut, 0.05, 100, 10, x0 = np.array([1,1]))
samples = x[0]
ess_values = {}
ess = {}
for method in samples.keys():
    ess_values[method] = np.mean(samples[method].compute_ess())
print(ess_values)
df_dict = {}
df_dict["mean"] =  [ess_values[method] for method in ["MH_fixed", "CWMH", "ULA", "MALA", "NUTS"]]
df_dict

Sample 110 / 110

Average acceptance rate: 0.8 

Sample 110 / 110

Average acceptance rate all components: 0.73 

Sample 110 / 110
Sample 110 / 110
Sample 110 / 110
{'MH_fixed': 4.170732420458758, 'CWMH': 1.5227487960045263, 'ULA': 17.93540562886835, 'MALA': 6.918327359366973, 'NUTS': 29.984546835854193}


{'mean': [4.170732420458758,
  1.5227487960045263,
  17.93540562886835,
  6.918327359366973,
  29.984546835854193]}

In [13]:
df = pd.DataFrame(df_dict)


In [21]:
Ns_MH_adapted = 10000
Nb_MH_adapted = 2000

MH_adapted_samples = MH_sampler.sample_adapt(Ns_MH_adapted, Nb_MH_adapted)
ess_MH_adapted = MH_adapted_samples.compute_ess()

Sample 12000 / 12000

Average acceptance rate: 0.2378 MCMC scale: 0.6590233107155901 



array([1.29030777, 1.09300641])

In [50]:
scale_ULA = 0.065
ULA_sampler = ULA(target=target_donut, scale=scale_ULA, x0=np.array([0,0]))
Ns_ULA  = 10000
ULA_samples = ULA_sampler.sample(Ns_ULA)
ess_ULA = ULA_samples.compute_ess()

Sample 10000 / 10000


In [18]:
MH_fixed_samples.compute_rhat(ULA_samples)

array([1.20546084, 1.13999197])

In [40]:
#x_uni = Gaussian(0, 1)
scale_MALA = 0.01
MALA_uni = MALA(target= target_donut, scale=scale_MALA, x0=np.array([0,0]))

Ns_MALA = 40000
MALA_samples_uni = MALA_uni.sample(Ns_MALA)
ess_MALA = MALA_samples_uni.compute_ess()

Sample 40000 / 40000


In [47]:
NUTS_donut = NUTS(target=target_donut, x0=np.array([0,0]))
Ns_NUTS = 3000
Nb_NUTS = 2
NUTS_donuts_samples = NUTS_donut.sample(Ns_NUTS, Nb_NUTS)
ess_NUTS = NUTS_donuts_samples.compute_ess()

Sample 3002 / 3002


In [48]:
import pandas as pd


def safe_access(array, index):
    return round(array[index], 3) if len(array) > index else None

ess_df = pd.DataFrame({
    "Sampling Method": ["MH_fixed", "MH_adapted", "ULA", "MALA", "NUTS"],
    "No. of Samples": [Ns_MH_fixed, Ns_MH_adapted, Ns_ULA, Ns_MALA, Ns_NUTS],
    "No. of Burn-ins": [Nb_MH_fixed, Nb_MH_adapted, None, None, None],
    "Scaling Factor": [scale_MH_fixed, scale_MH_fixed, scale_ULA, scale_MALA, None],
    "ESS (v0)":  [safe_access(ess_MH_fixed, 0), safe_access(ess_MH_adapted, 0), safe_access(ess_ULA, 0), safe_access(ess_MALA, 0), safe_access(ess_NUTS, 0)],
    "ESS (v1)": [safe_access(ess_MH_fixed, 1), safe_access(ess_MH_adapted, 1), safe_access(ess_ULA, 1), safe_access(ess_MALA, 1), safe_access(ess_NUTS, 1)]
})

# Optional: Replace None values with "-"
ess_df = ess_df.fillna("-")

# Display the DataFrame without the index
ess_df


,Sampling Method,No. of Samples,No. of Burn-ins,Scaling Factor,ESS (v0),ESS (v1)
0,MH_fixed,10000,0.0,0.05,10.021,1.657
1,MH_adapted,8500,1500.0,0.05,26.709,3.575
2,ULA,1000,-,0.065,6.610,4.743
3,MALA,40000,-,0.01,9.975,24.309
4,NUTS,3000,-,-,1086.525,960.139
